# 0. 폴더 경로 셋팅 및 기타 환경 변수 설정

## 날짜 별로 폴더 구성
```bash
2023-03-31
├── video (영상이 다운로드 되는 폴더)
├── images (로또 용지 폴더)
│   ├── tmp (분류 전 임시 저장 폴더) 
│   ├── success (자동 입력 성공 케이스)
│   ├── fail  (자동 입력 실패 케이스)
├── feedback (학습용 피드백 폴더)
└── result.txt
``` 

In [18]:
import os

root_dir = os.path.join("2023-03-31")
video_dir = os.path.join(root_dir, "video")
images_dir = os.path.join(root_dir, 'images')
images_success_dir = os.path.join(images_dir, "success")
images_fail_dir = os.path.join(images_dir, "fail")
images_tmp_dir = os.path.join(images_dir, "tmp")
feedback_dir = os.path.join(root_dir, 'feedback')

In [19]:
# 폴더 생성
dir_list = [root_dir, 
            video_dir, 
            images_dir, 
            images_success_dir, 
            images_fail_dir,
            images_tmp_dir,
            feedback_dir]

for dirpath in dir_list:    
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

# 1. 로또 영상 다운로드 및 로또 용지 추출

In [27]:
import cv2
import numpy as np

def compareImage(image1, image2):    
    diff = cv2.absdiff(image1, image2)
    diff_mean = np.mean(diff)
    return diff_mean

In [31]:
import pytube

def downloadVideo(name, url):
    yt = pytube.YouTube(url)
    video = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

    # 동영상 다운로드
    video.download(filename=f'{video_dir}/{name}.mp4')

In [32]:
def saveImage(name):
    cap = cv2.VideoCapture(f'{video_dir}/{name}.mp4')
    
    ret, frame1 = cap.read()
    count = 1

    while True:
        # 현재 프레임 읽기
        ret, frame2 = cap.read()

        # 더 이상 읽을 프레임이 없으면 루프 종료
        if not ret:
            break

        result = compareImage(frame1, frame2)

        # 변경된 부분이 있는 경우 이미지 저장
        if result > 5:
            cv2.imwrite(f"{images_tmp_dir}/{name}{count:02}.jpg", frame2)
            count += 1

        # 이전 프레임 업데이트
        frame1 = frame2
        
    cap.release()

In [98]:
# 작업할 youtube url list
youtube_list = [
#     {"name" : "1061-Mon", "url" : "https://www.youtube.com/watch?v=0FAUpNU_PwY"},
#     {"name" : "1061-Tus", "url" : "https://www.youtube.com/watch?v=Asn2Mdizm2I"},
#     {"name" : "1061-Wed", "url" : "https://www.youtube.com/watch?v=cNV9DlTK39U"},
#     {"name" : "1061-Thu", "url" : "https://www.youtube.com/watch?v=T8tckeCDw4s"},
    {"name" : "1061-Fri", "url" : "https://www.youtube.com/watch?v=StvxY-tnMDE"}
]

for info in youtube_list:
    name = info["name"]
    url = info["url"]
    
    # 동영상 다운로드
    print(f"{name} 다운로드 시작")
    downloadVideo(name,url)
    print(f"{name} 다운로드 완료")
    
    print(f"{name} 이미지 분리 시작")
    saveImage(name)
    print(f"{name} 이미지 분리 완료")

1061-Fri 다운로드 시작
1061-Fri 다운로드 완료
1061-Fri 이미지 분리 시작
1061-Fri 이미지 분리 완료


# 2. 로또 용지에서 번호 추출

In [99]:
def calc_distance(rect1, rect2):
    # rect1, rect2: [x, y, w, h]

    # 두 사각형의 중심점 좌표 계산
    rect1_center_x = rect1[0] + rect1[2] / 2
    rect1_center_y = rect1[1] + rect1[3] / 2
    rect2_center_x = rect2[0] + rect2[2] / 2
    rect2_center_y = rect2[1] + rect2[3] / 2

    # 두 사각형의 중심점 좌표간의 거리 계산
    distance = ((rect1_center_x - rect2_center_x) ** 2 + (rect1_center_y - rect2_center_y) ** 2) ** 0.5

    return distance

In [100]:
def is_overlap(rect1, rect2):
    # rect1, rect2: [x, y, w, h]

    # x, y, w, h 값 중에 어느 하나라도 겹치지 않는 경우 두 사각형은 겹치지 않음
    if rect1[0] + rect1[2] < rect2[0] or rect2[0] + rect2[2] < rect1[0] or \
            rect1[1] + rect1[3] < rect2[1] or rect2[1] + rect2[3] < rect1[1]:
        return False
    else:
        return True

In [101]:
def img_resize(input_img):
    width, height = 50, 50
    
    # 원하는 크기로 이미지 크기 조정
    resized_img = cv2.resize(input_img, (width, height))
    
    # 빈 이미지 생성
    background = np.zeros((height, width), dtype=np.uint8)
    
    # 이미지 중앙 좌표 계산
    x_offset = (width - resized_img.shape[1]) // 2
    y_offset = (height - resized_img.shape[0]) // 2

    # 이미지 중앙에 리사이즈된 이미지 삽입
    background[y_offset:y_offset+resized_img.shape[0], x_offset:x_offset+resized_img.shape[1]] = resized_img

    # 바깥 영역을 흰색으로 채우기
    background[0:y_offset, :] = 255
    background[y_offset+resized_img.shape[0]:, :] = 255
    background[:, 0:x_offset] = 255
    background[:, x_offset+resized_img.shape[1]:] = 255
    
    return background

In [102]:
# 번호 추출이 가능한 이미지인지 체크하는 함수
def is_available_image(path, view=False):
    img = cv2.imread(path)

    x, y, w, h = 265, 360, 1280, 720
    img = img[y:y+h, x:x+w]
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    
    mser = cv2.MSER_create()
    regions, _ = mser.detectRegions(gray)
    
    # 사각형 추출
    hulls = [cv2.convexHull(p.reshape(-1,1,2)) for p in regions]
    
    
    # 유효한 사각형 찾기 (step 1)
    tmp_result_step1 = []
    
    for i, tmp in enumerate(hulls):
        x, y, w, h = cv2.boundingRect(tmp)
        if (h < 20):
            continue
        tmp_result_step1.append([x,y,w,h])
        
    # 겹치는 사각형 찾기 (step 2)
    
    tmp_result_step2 = {}

    for i, tmp1 in enumerate(tmp_result_step1):
        tmp_result_step2[i] = []
        for j, tmp2 in enumerate(tmp_result_step1):
            # 사각형이 겹치는지 
            # 겹친다면, 큰 영역으로 데이터 교체, 진행
            overlap = is_overlap(tmp1, tmp2)
            if overlap:
                tmp_result_step2[i].append(j)
                
    # 중복 제거 (step 3)
    tmp_result_step3 = list(set(map(tuple,tmp_result_step2.values())))
    
    # 겹치는 사각형 하나로 만들기 (step 4)
    tmp_result_step4 = []
    
    for t in tmp_result_step3:
        x1, y1, x2, y2 = 1e9, 1e9, -1e9, -1e9
        for j in t:
            x1 = min(x1, tmp_result_step1[j][0])
            y1 = min(y1, tmp_result_step1[j][1])

            x2 = max(x2, tmp_result_step1[j][0] + tmp_result_step1[j][2])
            y2 = max(y2, tmp_result_step1[j][1] + tmp_result_step1[j][3])
        tmp_result_step4.append([x1,y1,x2-x1,y2-y1])
    
    #  거리가 가까운 사각형 찾기 (step 5)
    tmp_result_step5 = {}
    
    for i, tmp1 in enumerate(tmp_result_step4):
        tmp_result_step5[i] = []
        for j, tmp2 in enumerate(tmp_result_step4):
            if abs(tmp1[1] - tmp2[1]) < 4 and calc_distance(tmp1, tmp2) < 61:
                tmp_result_step5[i].append(j)
                
    # 중복 제거 (step 6)
    tmp_result_step6 = list(set(map(tuple,tmp_result_step5.values())))
    
    # 사각형 하나로 만들기 (step 7)
    tmp_result_step7 = []
    for t in tmp_result_step6:
        x1, y1, x2, y2 = 1e9, 1e9, -1e9, -1e9
        for j in t:
            x1 = min(x1, tmp_result_step4[j][0])
            y1 = min(y1, tmp_result_step4[j][1])

            x2 = max(x2, tmp_result_step4[j][0] + tmp_result_step4[j][2])
            y2 = max(y2, tmp_result_step4[j][1] + tmp_result_step4[j][3])
        tmp_result_step7.append([x1,y1,x2-x1,y2-y1])
    
    # 윗줄부터 숫자 추출 가능하게 정렬 (step 8)
    tmp_result_step8 = sorted(tmp_result_step7, key=lambda x:(x[1]))
    
    if view:
        clone = img.copy()
        for i, rect in enumerate(tmp_result_step8):
            x, y, w, h = rect
            cv2.rectangle(clone, (x,y), (x+w,y+h), (0,255,0),1)
            cv2.putText(clone, str(i), (x-2,y-2), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2)

        cv2.imshow('image',clone)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    return [len(tmp_result_step8) == 30, gray, tmp_result_step8]

In [103]:
from tensorflow.keras.models import load_model

model = load_model('lotto_model')
classname = ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '5', '6', '7', '8', '9']
# 숫자 예측
def predict_number(image):
    global model
    global classname
    predictions = model.predict(image)
    predicted_labels = np.argmax(predictions, axis=1)
    return classname[int(predicted_labels)]

In [134]:
import shutil

# 번호 추출
def extract_number(filename):
    (available, gray, result) = is_available_image(os.path.join(images_tmp_dir, filename), False)
    
    if not available:
        shutil.move(os.path.join(images_tmp_dir, filename), os.path.join(images_fail_dir, filename))
        return []
        
    clone = gray.copy()
    
    numbers = []
    # 번호 예측
    for i in range(5):
        number = []
        for j in range(6):
            id = i*6 + j
            x,y,w,h = result[id]
            margin = 2

            tmp_img = gray[y-margin:y+h+margin, x-margin:x+w+margin]
            tmp_img = cv2.medianBlur(tmp_img, 3)
            tmp_img = cv2.GaussianBlur(tmp_img, (5,5), 0)
            thresh = cv2.threshold(tmp_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            
            result_img = img_resize(thresh)
            rgb_image = cv2.cvtColor(result_img, cv2.COLOR_GRAY2RGB)
            input_img = np.expand_dims(rgb_image, axis=0)
            input_img = input_img / 255.0
            final_predict_number = int(predict_number(input_img))
            number.append(final_predict_number)
            cv2.rectangle(clone, (x,y), (x+w,y+h), (0,255,0),1)
            cv2.putText(clone, str(final_predict_number), (x+w-15,y+h), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0),2)
            
        numbers.append(list(map(str,sorted(number))))

    cv2.imwrite(os.path.join(feedback_dir, filename), clone)
    
    if available:
        shutil.move(os.path.join(images_tmp_dir, filename), os.path.join(images_success_dir, filename))

    return numbers

In [136]:
# final

f = open(os.path.join(root_dir,"23-03-31.txt"), "a")
fail_list = []

for filename in os.listdir(images_tmp_dir):
    result = extract_number(filename)
    
    if len(result) == 0:
        fail_list.append(filename)
        
    f.write("\n".join([",".join(i) for i in result]))
    f.write("\n")
    
f.close()

1/1 [==============================] - 0s 15ms/step
